## Quantum State Representation using Artifical Neural Networks

Here, we will use a Restricted Boltzmann Machine (or some other appropriate architecture along the way) to represent quantum states. Specifically, we aim to represent the ground state of a quantum many-body system. This obeys the transverse-field Ising model w/ Hamiltonian. 

$\hat{H} = -\sum^{N}_{i=1} \hat{\sigma}^{z}_{i} \hat{\sigma}^{z}_{i+1} - h \sum_{i=1}^{N} \hat{\sigma}^{x}_{i}$

Where $\hat{\sigma}_{i}^{z}$ and $\hat{\sigma}_{i}^{x}$ are Pauli operators acting on the $i^{th}$ qubit. Here, we can assume that we're normalized wrt the coupling constant $J$, so that the transverse field strength is really in terms of $J$. 


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import rbm # This implements the restricted boltzmann machine. 

Define some constants 

In [9]:
# Pauli operators

sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0, -1j], [1j, 0]])
sigma_z = np.array([[1, 0], [0, -1]])

# Initial parameters 
h=1 # Transverse field stength
N=2 # Size of quantum system 


### Dataset Generation 

Our first step is to generate data to train our RBM. We could start by generating the Hamiltonian. 

In [10]:
def compute_pauli_i(N, i, pauli_type='z'): # Note that i here works in base zero indexing 
    '''
    N - size of many body system
    i - which qubit are we acting on? 
    pauli_type - which pauli operator are we simulating?
    '''
    pauli_op = np.eye(2) # Or sigma 0
    
    if (pauli_type=='z'):
        pauli_op = sigma_z
    elif (pauli_type=='x'):
        pauli_op = sigma_x
    
    # The initialization of the pauli i operator depends on the value of i
    
    if (i==0):
        pauli_i = np.kron(pauli_op, np.eye(2))
    else:
        pauli_i = np.kron(np.eye(2), np.eye(2))
        
    #print(pauli_i)
    #input()
            
    for ii in range(N-2): # So we are starting from the ii = 2 position, not ii==0
        if (ii+2==i):
            pauli_i = np.kron(pauli_i, pauli_op)
        else:
            pauli_i = np.kron(pauli_i, np.eye(2))
        #print(pauli_i)
        #input()
        
    return pauli_i

def init_hamil(N): # This instantiates the 2^N by 2^N Hamiltonian we are going to generate
    '''
    N - number of qubits in the system 
    '''
    # Construct the 2^N by 2^N dimensional Hamiltonian. The initial Hamiltonian is a tensor product of N zero matrices.  

    hamil = np.zeros((2,2), dtype=np.complex128)
    
    for ii in range(N-1): # Since we have already initialized the first element in the tensor product. 
        temp = np.zeros((2,2), dtype=np.complex128)
        hamil = np.kron(hamil, temp)
    
    return hamil
    
        
def compute_hamiltonian(N, h):
    '''
    N -- Hamiltonian 
    h -- transverse field strengh 
    '''
    # Construct the 2^N by 2^N dimensional Hamiltonian. The initial Hamiltonian is a tensor product of N zero matrices.  

    hamil = init_hamil(N)
    
    # Now we compute the sum using the formula 
    
    #print(np.shape(compute_pauli_i(N, 0, pauli_type='z')))
    #print(np.shape(compute_pauli_i(N, 1, pauli_type='z')))
    #input()
    
    for ii in range(N):
        if (ii+1==N):
            hamil += -compute_pauli_i(N, ii, pauli_type='z')@compute_pauli_i(N, 0, pauli_type='z') - h*compute_pauli_i(N, ii, pauli_type='x')
        else: 
            hamil += -compute_pauli_i(N, ii, pauli_type='z')@compute_pauli_i(N, ii+1, pauli_type='z') - h*compute_pauli_i(N, ii, pauli_type='x')
        
    return hamil

#compute_pauli_i(2, 1, pauli_type='z')
            
hamil = compute_hamiltonian(2,1)

#print(hamil)
print(hamil)

[[-3.+0.j  0.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j -3.+0.j  0.+0.j -1.+0.j]
 [-1.+0.j  0.+0.j  1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.+0.j  1.+0.j]]


Diagonalize the Hamiltonian to retrieve the exact ground state. In the diagonalization, this is the eigenvector which has the lowest eigenvalue ... unless h=1, where we actually have degenerate ground states! This seems to be invariant of the size of the quantum system. 


In this case, a valid ground state could be some probalistic superposition between the two probable ground states. We'll fix $\alpha = \beta = \frac{1}{\sqrt{2}}$ 

In [11]:
from numpy.linalg import eigh

gnomeState = np.zeros(N, dtype=np.complex128)
eigSpectrum, eigSol = eigh(hamil)

if (h==1):
    groundState1, groundState2 = eigSol[np.argmin(eigSpectrum)], eigSol[np.argmin(eigSpectrum) + 1]
    
    # Let's create a ground state out of the ground states. Let's fix \alpha = \beta = 1/\sqrt{2}
    
    gnomeState = (1/np.sqrt(2))*(groundState1 + groundState2)

else: 
    gnomeState = eigSol[np.argmin(eigSpectrum)]
    
# Check normalization of the ground state

print(np.sum(np.abs(gnomeState)**2))

0.9999999999999998


We will also need to compute the basis states in the $\hat{\sigma_{z}}$ basis. This will be necessary to compute the NRG Hamiltonian. 

In [14]:
from itertools import product
from functools import reduce

def create_basis_array(N):
    '''
    The idea is that we create the 2^N combination of possible basis state configurations. A 0 means spin down, 1 means spin up
    N - Size of quantum system 
    '''
    
    #N = 3  # Change this to whatever length you need
    combinations = list(product([0, 1], repeat=N))

    for ii in range(len(combinations)):
        combinations[ii] = np.array(combinations[ii])
    
    return combinations

 
def gen_basis_state(N, basis_array): 
    
    '''
    N - number of qubits in our system 
    '''
    
    #base_ket = np.zeros(2, dtype=np.complex128)
    
    base_kets = []

    # Initialize the complete basis state, which occupies the 2^N dimensional Hilbert space. 
    
    for ii in range(N):
        base_kets.append(np.zeros(2, dtype=np.complex128))
        
    # Now, we modify copies of the base ket by consulting the basis_array 
    
    #print(basis_array)
    #print(combo)
    #input()
    
    
    full_basis_array = []
    
    for combo in basis_array: 
        # Create a new copy of the base_kets 
        
        temp = np.copy(base_kets)
        #print(combo)
        #input()
    
        for ii in range(N):
            if (combo[ii]==0):
                temp[ii] = np.array([1,0], dtype=np.complex128)
            elif (combo[ii]==1):
                temp[ii] = np.array([0,1], dtype=np.complex128)
                
        # Finally, compute the kronecker product iteratively across all elements in the array
        
        kron_result =  reduce(np.kron, temp)
        
        #print(kron_result)
        #input()
        full_basis_array.append(kron_result)
        
    
    return full_basis_array
     
    #for ii in range(N):
    
your_basis = gen_basis_state(3, create_basis_array(3))

Define some functions to compute the NRG expectation value.

In [ ]:
def local_hamil(basis_states, m, hamil, psi_basis):
    
    '''
    Computes the local energy as a function on the m^{th} basis state.  
    
    basis_states -- set of all basis states 
    m -- Index of m^{th} basis state that is of interest
    hamil -- Hamiltonian  
    psi_basis --  set of probability amplitudes 
    '''
    
    bra_op = lambda ket: np.conjugate(np.transpose(ket))
    local_hamil = 0
    
    for ii in range(len(basis_states)):
        local_hamil += bra_op(basis_states[m])@hamil@basis_states[ii]*(psi_basis[ii]/psi_basis[m])
    
    return local_hamil 
    
    
def hamil_expect(basis_states, hamil, psi_basis):
    
    '''
    Computes the Hamiltonian expectatiom value
    
    basis_states -- set of all basis states 
    hamil -- Hamiltonian 
    psi_basis -- set of probability amplitudes 
    '''
    
    M = len(basis_states)
    hamil_expect = 0
    
    for m in range(M): 
        hamil_expect += local_hamil(basis_states, hamil, psi_basis)
    return (1/M)*(hamil_expect)

### RBM Architecture

We need to compute the gradient update with respect to the Hamiltonian expectation value. 

In [1]:
def compute_grad(rbm, M):
    
    '''
    Computes (approximately) the gradient of the Hamiltonian expectation value with respect to the trainable parameters of the RBM
    rbm -- our RBM to train
    M -- number of Gibbs sampling steps 
    '''
    
    # Using the RBM, perform Gibbs sampling M times. 
    
    visi_gibbs, hid_gibbs = rbm.gibb_sample(M)
    
    # Gradient vector. It is of length equal to the set of trainable parameters of the RBM. 
    
    len_bias_visible = len(rbm.bias_visi)
    len_bias_hidden = len(rbm.bias_hid)
    
    len_bias_weight = len_bias_visible*len_bias_hidden
    grad_vector = np.zeros(len_bias_visible+len_bias_hidden+len_bias_weight) # Importantly, the first len_bias_visible entries of the vector correspond to the number of bias entries in the visible neurons, 
    # the next len_bias_hidden entries....
    
    # Now, using the visible neuron configurations, appoximate the Gibbs sampling. 

    for mm in range(M):
        grad_vector[0:len_bias_visible] += 
        
        
        
        
        
        
    

SyntaxError: incomplete input (1581405990.py, line 24)

## Appendix

In [9]:
# This implements the contrastive divergence method for updating the Restricted Boltzmann Machine. Some changes will need to be made in order 

def contrastive_div(train_dataset, rbm, epochs=10, lr=0.01, batch_size = 5, verbose=0): 
    num_of_batches = int(len(train_dataset)/batch_size)
    start_time = time.time()
    # Pick visible configurations from the dataset batchwise
    for ii in range(epochs): 
        print("**********")
        print(f"Epoch {ii}")
        # We need to iterate over the training set. For now, we iterate over datapoints, not batches
        for b in range(num_of_batches): 
            v_batch = train_dataset[batch_size*b:batch_size*(b+1)] # the visible neuron configuration from the dataset
            h_stuff_one = [] # the hidden neuron configuration after having done one Gibbs sampling step 
            h_stuff_two = [] # after two Gibbs sampling step
            v_stuff = [] # Sampling the visible neuron config after one Gibbs sampling step. 
            
            for v in v_batch:
                # Get h by performing a Gibbs Sampling step 
                rbm.sample_new_h(v)
                # Store the hidden vector for later 
                h_stuff_one.append(rbm.hid)
                # Get v_temp by performing a Gibbs Sampling step 
                v_temp = rbm.sample_new_v()
                v_stuff.append(v_temp)
                # Using v_temp, sample a new h vector by performing a Gibbs Sampling step 
                rbm.sample_new_h(v_temp)
                h_stuff_two.append(rbm.hid)
            
            # The next step is to compute the updates on the weight parameters
            # grad_update(self, visi_list_0, hid_list_0, visi_list_1, hid_list_1, i,j, lr, expect='weight')
        
            rbm.grad_update(v_batch, np.array(h_stuff_one), np.array(v_stuff), np.array(h_stuff_two), lr)
            
            if(verbose==1):
                print(f"Weights: {rbm.weights} ")
                print(f"Visible Bias: {rbm.bias_visi}")
                print(f"Hidden Bias: {rbm.bias_hid}")
                
                